In [ ]:
import pystan

In [ ]:
stations = pd.read_csv('input/stations_tokai3ken_edited.csv', index_col=0)

In [ ]:
X = stations.drop(['飲食店事業所数', '路線ID'], axis=1)
y = stations['飲食店事業所数']

In [ ]:
X.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
N_train = X_train.shape[0]
M = X_train.shape[1]
Y_train = list(y_train.values)
N_test = X_test.shape[0]
X_test = list(X_test.values)

data = dict(
    N_train=N_train,
    M=M,
    X_train=X_train,
    Y_train=Y_train,
    N_test=N_test,
    X_test=X_test,
)

In [ ]:
model = '''
data {
    int N_train;
    int M;
    matrix<lower=0>[N_train, M] X_train;
    vector<lower=0>[N_train] Y_train;
    int N_test;
    matrix<lower=0>[N_test, M] X_test;
}

parameters {
    real b0;
    vector[M] b;
    real<lower=0> s;
    real<lower=0> s_b;
}

model {
    b0 ~ double_exponential(0, s_b);
    b ~ double_exponential(0, s_b);
    s ~ normal(0, 5);
    Y_train ~ normal(b0 + X_train*b, s);
}

generated quantities{
#    vector<lower=0>[N_test] predict;
    vector[N_test] predict;
    for (i in 1:N_test)
        predict[i] = normal_rng(b0 + X_test[i]*b, s);
}
'''

In [ ]:
fit = pystan.stan(model_code=model, data=data, chains=3, iter=1000, warmup=200, thin=1)

In [ ]:
fit

In [ ]:
fit.extract()['predict'][0]

In [ ]:
y_test

In [ ]:
predict = fit.extract()['predict'][0]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(predict, y_test, edgecolors=(0, 0, 0))
ax.plot([predict.min(), predict.max()], [predict.min(), predict.max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()

# いろいろ捨てる

In [ ]:
stations = pd.read_csv('input/stations_tokai3ken_edited.csv', index_col=0)

In [ ]:
X = stations.drop(['生徒学生数', '飲食店事業所数', '路線ID', '小売事業所数', '小売業年間商品販売額', '全産業事業所数', '買回り品(事業所数比率)', '最寄り品(事業所数比率)', '全産業従業者総数', '人口総数' , '男性人口' , '世帯数'], axis=1)
y = stations['飲食店事業所数']

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
#多治見、土岐、(瑞浪、)恵那、中津川をtestに移動
X_tono = pd.DataFrame(data=[X_train.loc['g_1189'], X_train.loc['g_1190'], X_train.loc['g_1194'], X_train.loc['g_1196']])
X_test = pd.concat([X_test, X_tono], axis=0)
X_train = X_train.drop(['g_1189', 'g_1190', 'g_1194', 'g_1196'], axis=0)
y_tono = pd.Series(data=[y_train.loc['g_1189'], y_train.loc['g_1190'], y_train.loc['g_1194'], y_train.loc['g_1196']],
                      index=['g_1189', 'g_1190', 'g_1194', 'g_1196'])
y_test = pd.concat([y_test, y_tono], axis=0)
y_train = y_train.drop(['g_1189', 'g_1190', 'g_1194', 'g_1196'], axis=0)

In [ ]:
N_train = X_train.shape[0]
M = X_train.shape[1]
Y_train = list(y_train.values)
N_test = X_test.shape[0]
X_test = list(X_test.values)

data = dict(
    N_train=N_train,
    M=M,
    X_train=X_train,
    Y_train=Y_train,
    N_test=N_test,
    X_test=X_test,
)

In [ ]:
model = '''
data {
    int N_train;
    int M;
    matrix<lower=0>[N_train, M] X_train;
    vector<lower=0>[N_train] Y_train;
    int N_test;
    matrix<lower=0>[N_test, M] X_test;
}

parameters {
    real b0;
    vector[M] b;
    real<lower=0> s;
    real<lower=0> s_b;
}

model {
    b0 ~ double_exponential(0, s_b);
    b ~ double_exponential(0, s_b);
    s ~ normal(0, 5);
    Y_train ~ normal(b0 + X_train*b, s);
}

generated quantities{
#    vector<lower=0>[N_test] predict;
    vector[N_test] predict;
    for (i in 1:N_test)
        predict[i] = normal_rng(b0 + X_test[i]*b, s);
}
'''

In [ ]:
fit = pystan.stan(model_code=model, data=data, chains=3, iter=1000, warmup=200, thin=1)

In [ ]:
fit

In [ ]:
fit.extract()['predict'][0]

In [ ]:
y_test

In [ ]:
predict = fit.extract()['predict'][0]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(predict, y_test, edgecolors=(0, 0, 0))
ax.plot([predict.min(), predict.max()], [predict.min(), predict.max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()

# 乗降客数

In [ ]:
stations = pd.read_csv('input/stations_tokai3ken_edited.csv', index_col=0)

In [ ]:
X = stations['乗降客数（日）']
y = stations['飲食店事業所数']

In [ ]:
#from sklearn import linear_model
#clf = linear_model.LinearRegression()
#clf.fit(X.values, y.values)

# 散布図
plt.scatter(X, y)
 
# 回帰直線
#plt.plot(X, clf.predict(X))

In [ ]:
X = stations['乗降客数（日）']/stations['乗降客数（日）'].max()
y = stations['飲食店事業所数']/stations['飲食店事業所数'].max()

In [ ]:
plt.scatter(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
#多治見、土岐、(瑞浪、)恵那、中津川をtestに移動
X_tono = pd.Series(data=[X_train.loc['g_1189'], X_train.loc['g_1190'], X_train.loc['g_1194'], X_train.loc['g_1196']],
                      index=['g_1189', 'g_1190', 'g_1194', 'g_1196'])
X_test = pd.concat([X_test, X_tono], axis=0)
X_train = X_train.drop(['g_1189', 'g_1190', 'g_1194', 'g_1196'], axis=0)
y_tono = pd.Series(data=[y_train.loc['g_1189'], y_train.loc['g_1190'], y_train.loc['g_1194'], y_train.loc['g_1196']],
                      index=['g_1189', 'g_1190', 'g_1194', 'g_1196'])
y_test = pd.concat([y_test, y_tono], axis=0)
y_train = y_train.drop(['g_1189', 'g_1190', 'g_1194', 'g_1196'], axis=0)

In [ ]:
N_train = X_train.shape[0]
Y_train = list(y_train.values)
N_test = X_test.shape[0]
X_test = list(X_test.values)

data = dict(
    N_train=N_train,
    X_train=X_train,
    Y_train=Y_train,
    N_test=N_test,
    X_test=X_test,
)

In [ ]:
model = '''
data {
    int N_train;
    vector<lower=0>[N_train] X_train;
    vector<lower=0>[N_train] Y_train;
    int N_test;
    vector<lower=0>[N_test] X_test;
}

parameters {
#    real<lower=0, upper=1> a;
    real<lower=0> a;
    real<lower=0> b;
    real<lower=0> s;
}

model {
    Y_train ~ normal(a + b*X_train, s);
}

generated quantities{
#    vector<lower=0>[N_test] predict;
    vector[N_test] predict;
    for (i in 1:N_test)
        predict[i] = normal_rng(a + b*X_test[i], s);
}
'''

In [ ]:
fit = pystan.stan(model_code=model, data=data, chains=3, iter=1000, warmup=200, thin=1)

In [ ]:
fit

In [ ]:
#fit.extract()['predict'][0]

In [ ]:
#y_test

In [ ]:
predict = fit.extract()['predict'][0]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(predict, y_test, edgecolors=(0, 0, 0))
ax.plot([predict.min(), predict.max()], [predict.min(), predict.max()], 'k--', lw=1)
#ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()

In [ ]:
#predict = np.log1p(predict)
#y_test = np.expm1(y_test)

fig, ax = plt.subplots()
ax.scatter(predict, y_test, edgecolors=(0, 0, 0))
ax.plot([predict.min(), predict.max()], [predict.min(), predict.max()], 'k--', lw=1)
#ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()

normalじゃなくcauchyにしたら、a,sが収束しなかった

X,yを0~1に正規化しないと、bが小さすぎて収束しなかった

正規化してnormalでやれば、パラメータ収束したけど、精度めちゃくちゃ悪い